In [39]:
# | default_exp data

In [40]:
# | exporti

import abc
from dataclasses import dataclass
from datetime import date, timedelta
from itertools import product

import numpy as np
import polars as pl
from sklearn.preprocessing import QuantileTransformer as SKQuantileTransformer

In [41]:
# | export


class DataLoader:
    """A class for loading and processing time series data with adjustable parameters.

    This class handles loading CSV data, computing returns, and calculating rolling variance.

    Attributes:
        max_records: Maximum number of records to keep (from the end of the dataset)
    """

    def __init__(self, max_records: int = None):
        self.max_records = max_records

    def load_data(self, path: str) -> pl.DataFrame:
        """Load and process time series data from a CSV file.

        Args:
            path: Path to the CSV file containing the data

        Returns:
            Processed DataFrame with returns and rolling variance
        """
        df = (
            pl.read_csv(path, try_parse_dates=True, infer_schema_length=None)
            .rename({"Date": "date", "Price": "price"})
            .sort("date")
        )
        df = df.with_columns(
            ret=pl.col("price") / pl.col("price").shift(1),
        )

        if self.max_records is not None:
            df = df[-self.max_records :]

        return df

In [42]:
# | exec: false
# Create a data loader with default parameters and load the data
data_loader = DataLoader(max_records=9000)
source_df = data_loader.load_data("./data/ng_daily.csv")
source_df.head()

date,price,ret
date,f64,f64
1997-01-07,3.82,null
1997-01-08,3.8,0.994764
1997-01-09,3.61,0.95
1997-01-10,3.92,1.085873
1997-01-13,4.0,1.020408


In [1]:
# | export


@dataclass
class DFFeature(abc.ABC):
    source_field: str
    feature_name: str
    requested_lag: int | None = None
    step_size: int = 1

    def fit(self, X) -> None:
        pass

    @abc.abstractmethod
    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        pass


@dataclass
class ZeroBasedMonth(DFFeature):
    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            (pl.col(self.source_field).dt.month() - 1).alias(self.feature_name)
        )


@dataclass
class QuantileTransformer(DFFeature):
    """A stateful quantile transformer using sklearn's QuantileTransformer."""

    n_quantiles: int = 1000
    output_distribution: str = "uniform"

    def __post_init__(self):
        self.scaler = SKQuantileTransformer(
            n_quantiles=self.n_quantiles,
            output_distribution=self.output_distribution,
            copy=False,
        )
        self._fitted = False

    def fit(self, df: pl.DataFrame) -> None:
        """Fit the QuantileTransformer on the source_field."""
        values = df[self.source_field].to_numpy().reshape(-1, 1)
        self.scaler.fit(values)
        self._fitted = True

    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        """Transform the source_field with fitted QuantileTransformer."""
        if not self._fitted:
            raise RuntimeError(
                "QuantileTransformer must be fitted before calling extract."
            )

        values = df[self.source_field].to_numpy().reshape(-1, 1)
        transformed = self.scaler.transform(values).flatten()

        return df.with_columns(pl.Series(name=self.feature_name, values=transformed))


@dataclass
class Variance(DFFeature):
    rolling_variance_window: int = 3

    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            pl.col(self.source_field)
            .rolling_var(self.rolling_variance_window)
            .clip(lower_bound=1e-4)
            .alias(self.feature_name)
        )


@dataclass
class Square(DFFeature):
    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            (pl.col(self.source_field) ** 2).alias(self.feature_name)
        )


@dataclass
class LogReturn(DFFeature):
    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(pl.col(self.source_field).log().alias(self.feature_name))


@dataclass
class Identity(DFFeature):
    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        return df


@dataclass
class Derivative(DFFeature):
    def extract(self, df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            pl.col(self.source_field).diff(n=self.step_size).alias(self.feature_name)
        )


@dataclass
class FeatureEngineer:
    """A class for creating lagged features from time series data.

    This class handles the creation of lagged (shifted) features that can be used for
    GARCH-like models and other time series forecasting tasks.

    Attributes:
        columns: List of column names to create lags for
        n_shifts: Number of lag periods to create
        drop_nulls: whether to drop the nulls after rolling window calculations
    """

    def __init__(
        self,
        transforms: list[DFFeature],
        n_shifts=3,
        drop_nulls: bool = True,
    ):
        """Initialize the FeatureEngineer.

        Args:
            columns: List of column names to create lags for (default: ['ret', 'var'])
            n_shifts: Number of lag periods to create (default: 3)
            drop_nulls: whether to drop the nulls after rolling window calculations
        """
        self.transforms = transforms
        self.columns = [t.feature_name for t in transforms]
        self.dtrans = {t.feature_name: t for t in transforms}
        self.n_shifts = n_shifts
        self.drop_nulls = drop_nulls
        self.fitted = False

    def create_features(self, df: pl.DataFrame) -> pl.DataFrame:
        """Create lagged features from the input DataFrame.

        Args:
            df: Input DataFrame containing time series data

        Returns:
            DataFrame with original columns plus lagged features
        """
        # Create a copy of the dataframe to avoid modifying the original
        for t in self.transforms:
            if not self.fitted:
                t.fit(df)
            df = t.extract(df)

        if not self.fitted:
            self.fitted = True
        result_df = df.clone()

        # Create lagged features for each specified column
        for t in self.transforms:
            # Check if column exists in dataframe
            col = t.feature_name
            if col not in df.columns:
                print(f"Warning: Column '{col}' not found in dataframe. Skipping.")
                continue

            # Create each lag
            lags = t.requested_lag if t.requested_lag is not None else self.n_shifts
            for shift in range(1, lags + 1):
                # Create new column name (e.g., prev_ret_1, prev_var_2)
                new_col_name = f"prev_{col}_{shift}"

                # Add the shifted column to the dataframe
                result_df = result_df.with_columns(
                    pl.col(col).shift(shift * t.step_size).alias(new_col_name)
                )
        if self.drop_nulls:
            result_df = result_df.drop_nulls()
        return result_df

    def to_numpy_dict(self, df: pl.DataFrame, drop: set[str] | None = None) -> dict:
        """Convert the dataframe with lagged features to a dictionary of NumPy arrays.

        This method extracts the original columns that were used to create lags,
        as well as all the generated lag columns, and converts them to NumPy arrays.
        The resulting dictionary can be used directly with NumPyro models.

        Args:
            df: DataFrame with lagged features created by create_features()

        Returns:
            Dictionary mapping column names to NumPy arrays
        """
        if drop is None:
            drop = set()
        # Create a new dataframe with the features we want to process
        features_df = df.clone()

        # Dictionary to store the NumPy arrays
        numpy_dict = {}

        # Add original columns
        for col in self.columns:
            if col in drop:
                continue
            if col in features_df.columns:
                numpy_dict[col] = features_df[col].to_numpy()

        # Add lagged features
        for col in self.columns:
            if col in drop:
                continue
            for shift in range(1, self.n_shifts + 1):
                lag_col = f"prev_{col}_{shift}"
                if lag_col in features_df.columns:
                    numpy_dict[lag_col] = features_df[lag_col].to_numpy()

        return numpy_dict

    def get_iterator(self, site: str | None = None):
        if site is None:
            site = self.columns
        else:
            site = [site]
        for s in site:
            this_site_shifts = self.dtrans[s].requested_lag or self.n_shifts
            for shift in range(1, this_site_shifts + 1):
                yield (s, shift)

    def get_shift_pattern(self, site: str, shift: int, prefix: str = ""):
        return f"{prefix}prev_{site}_{shift}"

NameError: name 'dataclass' is not defined

In [44]:
# | exec: false
feature_engineer = FeatureEngineer(
    transforms=[
        LogReturn(source_field="ret", feature_name="log_ret"),
        Variance(source_field="price", feature_name="var", requested_lag=0),
        QuantileTransformer(
            source_field="var", feature_name="var_quantile", requested_lag=0
        ),
    ],
    n_shifts=3,
)
df_with_features = feature_engineer.create_features(source_df)
df_with_features.head()

date,price,ret,log_ret,var,var_quantile,prev_log_ret_1,prev_log_ret_2,prev_log_ret_3
date,f64,f64,f64,f64,f64,f64,f64,f64
1997-01-13,4.0,1.020408,0.020203,0.042433,0.860861,0.082384,-0.051293,-0.005249
1997-01-14,4.01,1.0025,0.002497,0.002433,0.316111,0.020203,0.082384,-0.051293
1997-01-15,4.34,1.082294,0.079083,0.037433,0.847429,0.002497,0.020203,0.082384
1997-01-16,4.71,1.085253,0.081814,0.122633,0.944662,0.079083,0.002497,0.020203
1997-01-17,3.91,0.830149,-0.186151,0.1603,0.95773,0.081814,0.079083,0.002497


In [45]:
# | exec: false
qt = QuantileTransformer(source_field="price", feature_name="price_quantile")
qt.fit(source_df)
source_df = qt.extract(source_df)
source_df.head()

date,price,ret,price_quantile
date,f64,f64,f64
1997-01-07,3.82,null,0.573073
1997-01-08,3.8,0.994764,0.569069
1997-01-09,3.61,0.95,0.535536
1997-01-10,3.92,1.085873,0.591091
1997-01-13,4.0,1.020408,0.607107


In [ ]:
# | export
def append_from_log_ret(
    df: pl.DataFrame,
    new_log_ret: float,
    inherit_vals: list[str],
    add_variables: dict[str, float],
) -> pl.DataFrame:
    """Adds a new record to the dataframe based on a log return value.

    Args:
        df: Input DataFrame containing time series data
        new_log_ret: The new log return value to add

    Returns:
        DataFrame with a new row appended
    """
    # Get the latest date and add one day
    last_date = df["date"].max()
    new_date = last_date + timedelta(days=1)

    # Calculate the new return value from log return
    new_ret = np.exp(new_log_ret)

    # Get the last price and calculate the new price
    last_price = df["price"].tail(1).item()
    new_price = last_price * new_ret
    d = {
        "date": [new_date],
        "price": [new_price],
        "ret": [new_ret],
    }
    for c, v in add_variables.items():
        d[c] = [v]

    for c in inherit_vals:
        d[c] = [df[c].tail(1).item()]
    # Create a new row
    new_row = pl.DataFrame(d)

    # Append the new row to the existing DataFrame
    return pl.concat([df, new_row])

In [47]:
# | export
def binary_feature_from_date_ranges(
    date_range: tuple[date, date],
    periods: list[tuple[date, date]],
    feature_name: str = "feature",
) -> pl.DataFrame:
    # Create a date range dataframe
    df = pl.DataFrame(
        {
            "date": pl.date_range(
                start=date_range[0], end=date_range[1], interval="1d", eager=True
            )
        }
    )

    # Start with all zeros
    df = df.with_columns(pl.lit(0).alias(feature_name))

    # For each period, set the feature to 1 if the date is within the range
    for start, end in periods:
        df = df.with_columns(
            pl.when((pl.col("date") >= start) & (pl.col("date") <= end))
            .then(1)
            .otherwise(pl.col(feature_name))
            .alias(feature_name)
            .cast(pl.Int64)
        )

    return df

In [48]:
# | eval: false

binary_feature_from_date_ranges(
    date_range=(date(2010, 1, 1), date(2026, 1, 1)),
    periods=[
        (date(2022, 2, 24), date(2026, 1, 1)),
    ],
    feature_name="RU/UA_war",
)

date,RU/UA_war
date,i64
2010-01-01,0
2010-01-02,0
2010-01-03,0
2010-01-04,0
2010-01-05,0
…,…
2025-12-28,1
2025-12-29,1
2025-12-30,1
